In [1]:
#import required libraries
import sqlite3
import pandas as pd
from google.colab import drive
from tabulate import tabulate

#mount Google Drive
drive.mount('/content/drive')
db_path = "/content/drive/MyDrive/Applied_Data_Science/northwind.db"

Mounted at /content/drive


In [2]:
#connect to the Northwind database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

#fetch the tables names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

if not tables:
    print("No tables found in the database.")
else:
    print("Tables in the database:")

    for table in tables:
        table_name = table[0]
        print(f"\nTable: {table_name}")

        #fetch column info and attributes
        cursor.execute(f'PRAGMA table_info("{table_name}");')
        columns = cursor.fetchall()

        #print the tables, the columns and their attributes
        headers = ["Column ID", "Name", "Type", "NotNull", "Default Value", "Primary Key"]
        table_data = [[col[0], col[1], col[2], col[3], col[4], col[5]] for col in columns]
        print(tabulate(table_data, headers=headers, tablefmt="grid"))

Tables in the database:

Table: Categories
+-------------+--------------+---------+-----------+-----------------+---------------+
|   Column ID | Name         | Type    |   NotNull | Default Value   |   Primary Key |
+=============+==============+=========+===========+=================+===============+
|           0 | CategoryID   | INTEGER |         0 |                 |             1 |
+-------------+--------------+---------+-----------+-----------------+---------------+
|           1 | CategoryName | TEXT    |         0 |                 |             0 |
+-------------+--------------+---------+-----------+-----------------+---------------+
|           2 | Description  | TEXT    |         0 |                 |             0 |
+-------------+--------------+---------+-----------+-----------------+---------------+
|           3 | Picture      | BLOB    |         0 |                 |             0 |
+-------------+--------------+---------+-----------+-----------------+---------------+


In [4]:
#query to fetch all orders for year 2020
query_orders = """ SELECT o.OrderID, o.CustomerID, c.CompanyName, c.ContactName, o.OrderDate
                      FROM Orders AS o JOIN Customers AS c
                      ON o.CustomerID = c.CustomerID
                      WHERE o.OrderDate LIKE '%2020%'; """
orders_date = pd.read_sql_query(query_orders, conn)
print("Orders from year 2020")
orders_date_head = orders_date.head(25)
orders_date_head.set_index('OrderID', inplace=True)
print(tabulate(orders_date_head, headers='keys', tablefmt='grid'))

Orders from year 2020
+-----------+--------------+------------------------------------+-------------------------+---------------------+
|   OrderID | CustomerID   | CompanyName                        | ContactName             | OrderDate           |
+===========+==============+====================================+=========================+=====================+
|     11087 | WANDK        | Die Wandernde Kuh                  | Rita Müller             | 2020-04-25 14:00:46 |
+-----------+--------------+------------------------------------+-------------------------+---------------------+
|     11088 | PRINI        | Princesa Isabel Vinhos             | Isabel de Castro        | 2020-10-08 17:17:24 |
+-----------+--------------+------------------------------------+-------------------------+---------------------+
|     11140 | Val2         | IT                                 | Val2                    | 2020-08-27 13:25:15 |
+-----------+--------------+------------------------------------+-

In [5]:
#query to fecth top 5 selling products
query_sales = """ SELECT od.ProductID, p.ProductName, SUM(od.UnitPrice * od.Quantity) AS Total
              FROM "Order Details" AS od JOIN Products AS p
              ON od.ProductID = p.ProductID
              GROUP BY od.ProductID
              ORDER BY Total DESC LIMIT 5; """
total_sales = pd.read_sql_query(query_sales, conn)
total_sales.set_index('ProductID', inplace=True)
print("\nTop 5 Products by Total Sales")
print(tabulate(total_sales, headers='keys', tablefmt='grid'))


Top 5 Products by Total Sales
+-------------+-------------------------+-------------+
|   ProductID | ProductName             |       Total |
+=============+=========================+=============+
|          38 | Côte de Blaye           | 5.32745e+07 |
+-------------+-------------------------+-------------+
|          29 | Thüringer Rostbratwurst | 2.46308e+07 |
+-------------+-------------------------+-------------+
|           9 | Mishi Kobe Niku         | 1.94246e+07 |
+-------------+-------------------------+-------------+
|          20 | Sir Rodney's Marmalade  | 1.66549e+07 |
+-------------+-------------------------+-------------+
|          18 | Carnarvon Tigers        | 1.26075e+07 |
+-------------+-------------------------+-------------+


In [6]:
#query to find the highest-selling employee
query_employee = """ SELECT e.EmployeeID, e.FirstName, e.LastName, e.Title, COUNT(o.OrderID) AS Total
                        FROM Employees AS e JOIN Orders AS o
                        ON e.EmployeeID = o.EmployeeID
                        GROUP BY e.EmployeeID
                        ORDER BY Total DESC LIMIT 1; """
highest_employee = pd.read_sql_query(query_employee, conn)
highest_employee.set_index('EmployeeID', inplace=True)
print("\nHighest-Selling Employee")
print(tabulate(highest_employee, headers='keys', tablefmt='grid'))


Highest-Selling Employee
+--------------+-------------+------------+----------------------+---------+
|   EmployeeID | FirstName   | LastName   | Title                |   Total |
+==============+=============+============+======================+=========+
|            4 | Margaret    | Peacock    | Sales Representative |    1908 |
+--------------+-------------+------------+----------------------+---------+
